In [99]:
# public packages
import andes
import ams
import cvxpy as cp
import numpy as np
import gurobipy

# local packages
import anci_fun as af

# include dynamic constraints using reduced input/output NN model

# ------ nn information ------
# input:   [M, D]
# output:  [rocof_max, fnadir, dtheta_max, eig_max]
# nn_diam: [16, 128, 4]

# Para preparetion

In [100]:
# neural network parameters 
nn_name = 'trained_model.pth'
nn_weights, nn_bias, nn_diam = af.extract_nn_para(nn_name)

In [101]:
# MD parameters
vsg_num = 8

# lb_M = np.array([0, 0, 0, 0, 0, 0, 0, 0])
lb_M = np.array([0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
ub_M = np.array([6, 6, 6, 6, 6, 6, 6, 6])

# lb_D = np.array([0, 0, 0, 0, 0, 0, 0, 0])
lb_D = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])
ub_D = np.array([4, 4, 4, 4, 4, 4, 4, 4])


In [102]:
sf_load = 1.0

# y = [rocof_max, fnadir, dtheta_max, eig_max]
lb_y = np.array([-0.5, -0.6, -3.14, -1000000])
ub_y = np.array([ 0.2,  0.6,  3.14,  0])

# Load ams model

In [103]:
ssp = ams.load('ams_case123_REGCV1_v5.xlsx')

ssp.LDOPF.run()

ssp.LDOPF.om.mdl.solve()

ssp.LDOPF.om.vars

LDOPF solved as optimal in 0.0353 seconds with exit code 0.


OrderedDict([('pg', Variable((11,))),
             ('pn', Variable((117,))),
             ('qg', Variable((11,))),
             ('qn', Variable((117,))),
             ('vsq', Variable((117,))),
             ('pl', Variable((116,))),
             ('ql', Variable((116,)))])

In [104]:
# ssp.LDOPF.pg.get_idx()

# Define new variables, constraints, and obj

In [105]:
# M and D

M = cp.Variable(vsg_num, name="M")
D = cp.Variable(vsg_num, name="D")

constraints_M = [M >= lb_M, M <= ub_M]
constraints_D = [D >= lb_D, D <= ub_D]

In [106]:
# data for the neural network linearization
pg    = ssp.LDOPF.om.pg

x1    = cp.hstack([M, D])  # this version doesn't incelu load scaling factor

w1    = nn_weights[0]
w2    = nn_weights[1]

b1    = nn_bias[0]
b2    = nn_bias[1]

hdown = -100
hup   = 100

In [107]:
# neural network linization

z = cp.Variable(nn_diam[1], name="z")
a = cp.Variable(nn_diam[1], name="a", boolean=True)

z_ = w1 @ x1 + b1

y = w2 @ z + b2

constraints_nn = [
                    # linization constraints
                    z <= z_ - hdown * (1 - a),
                    z >= z_,
                    z <= hup * a,
                    z >= 0,
                    # output constraints
                    y[0] <= ub_y[0],         # RoCoF
                    y[0] >= lb_y[0],
                    y[1] <= ub_y[1],         # fnadir
                    y[1] >= lb_y[1],
                    y[2] <= ub_y[2],         # dtheta_max
                    y[2] >= lb_y[2],
                    y[3] <= ub_y[3],         # eig_max
                    # y[3] >= lb_y[3]             
                ]

In [108]:
y[0]

Expression(AFFINE, UNKNOWN, ())

In [109]:
# extract original ams constraints
constriants_ams = [ssp.LDOPF.om.constrs[cname] for cname in ssp.LDOPF.om.constrs.keys()]

In [110]:
constraints = constraints_M + constraints_D + constraints_nn + constriants_ams

# constraints = constriants_ams + constraints_M + constraints_D

# Formulate new problem

pg = ['PV_1',
 'PV_2',
 'PV_3',
 'PV_4',
 'PV_5',
 'PV_6',
 'PV_7',
 'PV_8',
 'PV_9',
 'PV_10',
 'PV_11']

 SG: PV_1, PV_2, PV_3 (slack bus)

 IBR: PV_4, PV_5, ..., PV_11

In [111]:
c2 = np.array([0.01, 0.01, 0.02, 0, 0, 0, 0, 0, 0, 0, 0])
c1 = np.array([  20, 20,   30,   0, 0, 0, 0, 0, 0, 0, 0])

# d  = np.array([0, 0, 0, 0.01, 0, 0, 0.001, 0])
c_M  = np.array([0.02, 0.01, 0.5, 0.01, 0.06, 0.07, 0.01, 0.02])

c_D  = np.array([0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.01])

pg_power = pg**2

cost = cp.sum(c2 @ pg_power + c1 @ pg + c_M @ M + c_D @ D)

obj = cp.Minimize(cost)

In [112]:
# Create the optimization problem
prob = cp.Problem(obj, constraints)

# Solve the optimization problem

# prob.solve(solver=cp.GUROBI, verbose=True)
prob.solve(solver=cp.GUROBI)

2.0589561075903093

In [113]:
prob.status

'optimal'

In [114]:
ssp.LDOPF.om.pg.value

array([0.02 , 0.02 , 0.04 , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.2  , 0.073])

In [115]:
M.value

array([0.05      , 0.05      , 0.05      , 0.12461236, 0.05      ,
       0.05      , 0.08709397, 0.32764103])

In [116]:
D.value

array([0.01      , 0.25098226, 0.01      , 0.57577362, 0.58418982,
       0.01      , 0.01      , 0.26367667])

# 